In [1]:
import pandas as pd
import numpy as np
import math
import glob
import datetime

In [2]:
"""
Подготовка списка датафреймов по соответствующим постам
И списка ID выбранных постов

"""

path = '/mnt/SSD/education/aspirantura/CAMELS_ru/files/hydrology/2020-09-22_15-39'
all_files = glob.glob(path + '/*_Q_Day.Cmd.txt')


In [ ]:
"""
Store Lat, Lon values as tuple with same
position as index of station
            
"""

GRDC_Stations = pd.read_excel(path + '/GRDC_Stations.xlsx')
Valid_lat_lon = list()
for j in range(len(Valid_WS)):
    for i in range(len(temporary)):    
        if Valid_WS.ID.loc[j] == temporary.grdc_no.loc[i]:
            Valid_lat_lon.append((temporary.lat.loc[i], temporary.long.loc[i]))

In [3]:
list_of_gauges = list()
list_of_station_IDS = [i[74:81] for i in all_files] # лист всех ID из представленных постов
valid_IDS = list()
for i, gauge in enumerate(all_files):
    
    """
    skip instance if it's empty
    
    """
    test = pd.read_csv(gauge, skiprows = 36, delimiter = ';', skipinitialspace = True)
    
    if test.empty:
        pass
    
    else:
        """
        read data; 
        change default column names;
        replace -999 with NaN;
        drop unnecessary column;
        add column with baseflow calculated with BFI function
        add it to list of valid time serieses 
        
        """
        test.Value = test.Value.replace(-999, np.NaN)
        test['YYYY-MM-DD'] = pd.to_datetime(test['YYYY-MM-DD'])
        test = test.drop(columns = ['hh:mm'])
        test.rename(columns = {'YYYY-MM-DD': 'Date', 'Value': 'Q'}, inplace = True)
        test['Qbase'] = np.NaN
        list_of_gauges.append(test)
        valid_IDS.append(int(list_of_station_IDS[i]))
        
print('количество постов с убранными пустышками {}, всего постов с суточными данными {}'.format(len(list_of_gauges), len(all_files)))

количество постов с убранными пустышками 211, всего постов с суточными данными 1491


In [4]:
"""
Чтение geojson
Создание списка атрибутов всех водосборов с площадью

"""

import geojson as gjs
with open(path + '/stationbasins.geojson') as f:
    WS_info = gjs.load(f)

WS_info = [i['attributes'] for i in WS_info['features']]

Valid_WS_Info = [WS_info[i] for i in range(len(WS_info)) if WS_info[i]['grdc_no'] in valid_IDS]

In [77]:
Valid_WS = pd.DataFrame(columns = {'ID', 'River_Name', 'WS_area'}) # dataframe for WS data
Valid_WS.ID = valid_IDS # assign only valuable WS with daily discharge available

temp_list_for_AREA = [[] for _ in range(len(Valid_WS.ID))] # empty list to store river name
# list of lists is used as a solution to solve .loc copy problem from df
temp_list_for_RIVER_NAME = [[] for _ in range(len(Valid_WS.ID))]

for j in range(len(Valid_WS.ID)):
    for i in range(len(Valid_WS_Info)):
        fnd = True
        while fnd:
            
            fnd = False
            
            if Valid_WS.ID.loc[j] == Valid_WS_Info[i]['grdc_no']:
                temp_list_for_AREA[j].append(Valid_WS_Info[i]['area_hys'])
                temp_list_for_RIVER_NAME[j].append(Valid_WS_Info[i]['river'] + ' - ' + Valid_WS_Info[i]['station'])
                
                fnd = True
                break
                
temp_list_for_AREA = [np.NaN if len(i) == 0 else i[0] for i in temp_list_for_AREA]
temp_list_for_RIVER_NAME = [np.NaN if len(i) == 0 else i[0] for i in temp_list_for_RIVER_NAME]

"""
Датасет сформирован таким образом, что здесь используются только те посты, 
по которым есть данные суточного стока
Расположены они в том же порядке, что и данные по расходам в
переменной

Далее, обращаясь к ID, получаем данные по площади водосбора,
которые используются дальше, при получении миллиметров слоя

"""

Valid_WS.River_Name = temp_list_for_RIVER_NAME
Valid_WS.WS_area = temp_list_for_AREA
Valid_WS = Valid_WS[[Valid_WS.columns.tolist()[1], Valid_WS.columns.tolist()[2], Valid_WS.columns.tolist()[0]]]

print('количество постов с суточными данными и площадью {}, всего постов с суточными данными {}'.format(len(Valid_WS.WS_area.dropna()), len(Valid_WS.ID.dropna())))

количество постов с суточными данными и площадью 186, всего постов с суточными данными 211


In [78]:
"""
Дальнейший расчёт будет вестись только для тех постов,
у которых есть данные о площади

"""
Valid_WS = Valid_WS.dropna().reset_index(drop = True)
Valid_gauges = [gauge for i, gauge in enumerate(list_of_gauges) if valid_IDS[i] in Valid_WS.ID.to_numpy()]

import copy

Valid_gauges_Q_mm = copy.deepcopy(Valid_gauges)

"""
calculate layer of discharge

"""

for i, gauge in enumerate(Valid_gauges_Q_mm):
    gauge.Q = 86400 * gauge.Q * 10**9 / (Valid_WS.WS_area.loc[i] * 10**12)
    


In [79]:
def split_by_year(list_of_something):
    
    """
    Данная функция разбивает лист датафреймов по годам
    и выкидывает года, где NaN больше, чем 2 месяца
    
    Колонка Date в датафрейме внутри листа - даты
    и она обязательна. 
    Столбцы могут называться в соответствии с пожеланиями
    
    """
    splitted_list = list()

    for i, gauge in enumerate(Valid_gauges):
        """
        From list of lists we are calling another list 
        that consist list of dataframes

        """
        year_split = list()
        unique_years = gauge.Date.dt.year.unique()

        for year in unique_years:
            year_split.append(gauge[gauge.Date.dt.year == year].reset_index(drop = True))

        for i in range(len(year_split)-1, -1, -1):
            if sum(year_split[i].Q.isna()) > len(year_split[i].Q)/6:
                del year_split[i]
            else:
                pass

        splitted_list.append(year_split)
    
    return splitted_list

every_gauge_split_by_year = split_by_year(Valid_gauges)

### q_mean

In [ ]:
"""
После фильтрации оказалось, что некоторые годы на постах имеют в себе Большое количество пропусков

"""
Valid_gauges_Q_mean_yearly = [np.mean([yearly_sum.Q.sum() for yearly_sum in gauge]) if len([yearly_sum.Q.sum() for yearly_sum in gauge]) > 2 else np.NaN for gauge in every_gauge_split_by_year]

print('количество постов с рассчитанным средним {}, всего постов с суточными данными {}'.format(
    len(np.array(Valid_gauges_Q_mean_yearly)[~np.isnan(Valid_gauges_Q_mean_yearly)]),
    len(Valid_gauges_Q_mean_yearly)))

### slope_fdc

In [ ]:
np.nanquantile(test.Q.to_numpy(), q = 0.33), np.nanquantile(test.Q.to_numpy(), q = 0.66), np.nanquantile(test.Q.to_numpy(), q = 0.99)

In [ ]:
try:
    (math.log(np.nanquantile(test.Q.to_numpy(), q = 0.33)) - math.log(np.nanquantile(test.Q.to_numpy(), q = 0.66)))/(0.66 - 0.33)
except ValueError:
    print('33 перцентиль = {}, 66 перцентиль = {}'.format(np.nanquantile(test.Q.to_numpy(), q = 0.33), np.nanquantile(test.Q.to_numpy(), q = 0.66)))
finally:
    print(
        (math.log(np.nanquantile(test.Q.to_numpy(), q = 0.33)+0.0009) - math.log(np.nanquantile(test.Q.to_numpy(), q = 0.66)+0.001))/(0.66 - 0.33))

In [ ]:
from numpy import random
import matplotlib.pyplot as plt
import seaborn as sns

sns.histplot(test.Q)

plt.show()

### baseflow_index

In [22]:
def clump_array(a):

    """
    Разбить период наблюдений на куски, в которых нет NaN
    """

    return [a[s] for s in np.ma.clump_unmasked(np.ma.masked_invalid(a))]

"Создаём отдельные вектора в соответствии с разбиением на куски без NaN"

###################################################################
def FirstPass(Q, alpha):

    q_f_1 = [[[] for _ in range(len(i))] for i in Q]
    q_b_1 = [[[] for _ in range(len(i))] for i in Q]

    for i in range(len(Q)):           
        """
        Задаём первые величины векторе "быстрого" стока

        """
        q_f_1[i][0] = Q[i][0]

        for j in range(len(Q[i])-1):
            """
            Для каждого куска считаем быстрый сток

            """
            q_f_1[i][j+1] = alpha * q_f_1[i][j] + 0.5 * (1 + alpha) * (Q[i][j+1] - Q[i][j])

        for j in range(len(Q[i])):
            if q_f_1[i][j] < 0:
                q_b_1[i][j] = Q[i][j]
            else:
                q_b_1[i][j] = Q[i][j] - q_f_1[i][j]

    Q_forward_1 = [q_f_1, q_b_1]

    return Q_forward_1

###################################################################

def BackwardPass(Q_forward_1, alpha):

    """
    Здесь Q - n-мерный лист в зависимости от числа разбиений
    """

    Qq = Q_forward_1[0]
    Qb = Q_forward_1[1]

    q_f_2 = [[[] for _ in range(len(i))] for i in Qq]
    q_b_2 = [[[] for _ in range(len(i))] for i in Qb]

    for i in range(len(Qq)):
        "последняя величина предыдущего шага - первая в обратном"
        q_f_2[i][-1] = Qb[i][-1]

        for j in range(len(Qq[i])-2, -1, -1):
            q_f_2[i][j] = alpha * q_f_2[i][j+1] + 0.5 * (1 + alpha) * (Qb[i][j] - Qb[i][j+1])

        for j in reversed(range(len(Qq[i]))):
            if q_f_2[i][j] < 0:
                q_b_2[i][j] = Qb[i][j]
            else:
                q_b_2[i][j] = Qb[i][j] - q_f_2[i][j]
    Q_backward = [q_f_2, q_b_2]

    return Q_backward

###################################################################

def ForwardPass(Q_backward, alpha):

    Qq = Q_backward[0]
    Qb = Q_backward[1]

    q_f_3 = [[[] for _ in range(len(i))] for i in Qq]
    q_b_3 = [[[] for _ in range(len(i))] for i in Qb]

    for i in range(len(Qq)):

        "Теперь первая величина предыдущего шага - первая и здесь"

        q_f_3[i][0] = Qb[i][0]

        for j in range(len(Qb[i])-1):

            q_f_3[i][j+1] = alpha * q_f_3[i][j] + 0.5 * (1 + alpha) * (Qb[i][j+1] - Qb[i][j])

        for j in range(len(Qb[i])):
            if q_f_3[i][j] < 0:
                q_b_3[i][j] = Qb[i][j]
            else:
                q_b_3[i][j] = Qb[i][j] - q_f_3[i][j]

    Q_forward = [q_f_3, q_b_3]

    return Q_forward

###################################################################

def BFI_calc(Q, alpha, passes, reflect):
    """
    we reflect the first reflect values and the last reflect values.  
    this is to get rid of 'warm up' problems © Anthony Ladson
    """ 
    
    Qin = Q
    
    "отложим отраженные величины с конца и начала нашего ряда"
    Q_reflect = [[[] for _ in range(len(i) + 2 * reflect)] for i in Q]

    for i in range(len(Q_reflect)):

        Q_reflect[i][:reflect] = Q[i][(reflect):0:-1] #добавляем 

        Q_reflect[i][(reflect):(reflect + len(Q[i]))] = Q[i]

        Q_reflect[i][(reflect + len(Q[i])):(len(Q[i]) + 2 + 2 * reflect)] = Q[i][len(Q)-2:len(Q) - reflect - 2:-1]
    
    Q1 = FirstPass(Q_reflect, alpha)

    "how many backwards/forward passes to we need © Anthony Ladson"

    n_pass = round(0.5 * (passes -1))

    BackwardPass(Q1, alpha)

    for i in range(n_pass):
        Q1 = ForwardPass(BackwardPass(Q1, alpha), alpha)

    ################# end of passes  ##############################
    
    Qbase = [i[reflect:(len(i)-reflect)] for i in Q1[1]]
    Qbase = [[0 if j < 0 else j for j in i] for i in Qbase]
    
    # расчёт взвешенного среднего для разбитого на куски ряда расходов
    
    len_Qbase = len([item for sublist in Qbase for item in sublist]) # длина всех расходов, без учёта пропусков
    weights_for_average = [len(i)/len_Qbase for i in Qbase] # вес для среднего по каждому элементу
    weighted_bfi = sum([weights_for_average[i] * np.mean(Qbase[i]/np.mean(Qin[i])) for i in range(len(Qbase))]) #взвешенное среднее по выпуклой комбинации

    return weighted_bfi, Qbase

### проверка на базовом датасете Anthony Ladson

In [ ]:
import requests
import csv
url = "https://raw.github.com/TonyLadson/data/master/data/BassRiver@Loch.csv"

with requests.Session() as s:
    download = s.get(url)

    decoded_content = download.content.decode('utf-8')

    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)

my_list = [int(i[0]) for i in my_list]

In [14]:
import random

alpha_coefficients = [random.uniform(0.9, 0.98) for i in range(1000)]

In [32]:
len(temp_for_bfi[0])

0

In [83]:
from tqdm import trange
temp_for_bfi = [[[] for _ in range(len(gauge))] for gauge in every_gauge_split_by_year]
temp_for_Qbase = [[[] for _ in range(len(gauge))] for gauge in every_gauge_split_by_year]

passes = 3
reflect = 30
import time

for j, gauge in enumerate(every_gauge_split_by_year):
    start_time = time.time()
    for k, year in enumerate(gauge):
        for alpha in alpha_coefficients:
            temp_for_bfi[j][k].append(BFI_calc(clump_array(year.Q.to_numpy()), alpha, passes = 3, reflect = 30)[0])
            temp_for_Qbase[j][k].append(BFI_calc(clump_array(year.Q.to_numpy()), alpha, passes = 3, reflect = 30)[1])
    stop = time.time()
    print('на расчёт итерации {} ушло {}'.format(j, stop-start_time))

на расчёт итерации 0 ушло -116.56409573554993
на расчёт итерации 1 ушло -185.62007665634155
на расчёт итерации 2 ушло -38.93612599372864
на расчёт итерации 3 ушло -167.1171293258667
на расчёт итерации 4 ушло -36.936737060546875
на расчёт итерации 5 ушло -32.5407919883728


KeyboardInterrupt: 

In [76]:
len(clump_array(every_gauge_split_by_year[j][k].Q)[0])

18

In [34]:

Q = clump_array(every_gauge_split_by_year[0][0].Q.to_numpy())


alpha = 0.98

bfi = BFI_calc(Q, 0.98, 3, 30)[0]

In [35]:
BFI_calc(clump_array(year), alpha_coefficients[i], passes = 3, reflect = 30)[0]

0.4943865494331915

In [ ]:
"""
Восстановления ряда baseflow с учётом ранее "выбитых" пропусков
"""
from itertools import groupby
Qbase_full = [list(group) for key, group in groupby(test.Q.to_numpy(), key=np.isnan)] # разбитие на группы по признаку. Получается n листов. NaN и !NaN разбиты по листам
mask_to_refill = [~np.isnan(i).any() for i in Qbase_full] # индексируем их как одинокие булевые величины

position_index = -1
for j, mask in enumerate(mask_to_refill):
    """
    Так как первичный разрезанный лист из функции clumped_array имеет в себе количество листов, такое же как и True, то восстанавливаем в соответствии
    
    Аргумент position_index подбирается в соответствии с присутсвием с True и адресуется к нужному куску в Qbase
    
    Всё перезаписывается в лист, где в перезаписываемых значениях эквивалентные по длине значения стока
    """
    if mask:
        position_index += 1
        Qbase_full[j] = Qbase[position_index]
    else:
        position_index = position_index
        Qbase_full[j] = Qbase_full[j]
            
Qbase_full = [item for sublist in Qbase_full for item in sublist] # лист листов в лист с учётом NaN

test.Qbase = Qbase_full # Добавляем его в исходный датафрейм

In [ ]:
split_by_year = [test[test['Date'].dt.year == y].reset_index(drop = True) for y in test['Date'].dt.year.unique()]
for i in split_by_year:
    i = i.set_index('Date')
    ax = i.plot()


### hfd_mean

In [ ]:
split_by_year = [test[test['Date'].dt.year == y].reset_index(drop = True) for y in test['Date'].dt.year.unique()]

hfd = list()

In [ ]:
for i in range(len(split_by_year)):
    half_discharge = 0
    for j, discharge in enumerate(split_by_year[i].Q):    
        half_discharge += discharge
        if half_discharge > split_by_year[i].Q.sum()/2:
            break
    hfd.append(split_by_year[i].Date[j])
    

In [ ]:
hfd